In [ ]:
import numpy as np

Initial value of the plasma parameters :
* $\rho_0$ the upstream charge density : `n0`
* $B_0$ the upstream modulus of the magnetic field : `B0`
* $\beta_0$ the upstream plasma parameter : `beta0`
* $\Theta_0$ the angle between the upstream magnetic field and the normal to the shock : `theta0`

In [ ]:
n0 = 1.0
B0 = 1.0
beta0 = 0.8
theta0 = 30*np.pi/180.0

We use `n` and `T` index notation for normal and tangential, respectively. Following the definition of `theta`

In [ ]:
B0_n = np.cos(theta0)
B0_T = np.sin(theta0)

From the definition of the $\beta$ parameter :

In [ ]:
p0 = 0.5*beta0*B0**2

One explore the range of upstream velocity normal to the shock

In [ ]:
v0_n_span = np.linspace(0.05, 2, 40)
print(v0_n_span)

We then have as unknown : `n0`, `n1`, `v0_n`, `v0_T`, `v1_n`, `v1_T`, `p0`, `p1`, `B0_n` `B0_T`, `B1_n`, `B1_T`, that is 12 unknowns.
We already know as parameter or initial range of values : `n0`, `v0_n`, `p0`, `B0_n` and `B0_T`, that is 5 unknowns.

We then need 7 equations from the RH jump equations :

**Maxwell-Thomson** : 1 equation

In [ ]:
# B1_n-B0_n = 0

**Maxwell-Faraday** (tangential) : 1 equation

In [ ]:
# v0_n*B0_T-B0_n*v0_T-v1_n*B1_T-B1_n*v1_T = 0

**de Hoffmann-Teller frame** : 1 equation

In [ ]:
# B0_n*v0_T-v0_n*B0_T = 0
# the second equation (just below) is then the same as given by Maxwell-Faraday
# B1_n*v1_T-v1_n*B1_T = 0

**Mass conservaton** : 1 equation

In [ ]:
# n0*v0_n-n1*v1_n = 0

**Momentum conservation** : 2 equations

In [ ]:
# n0*v0_n**2+p0+0.5*(B0_T**2-B0_n**2)-n1*v1_n**2+p1+0.5*(B1_T**2-B1_n**2) = 0
# n0*v0_n*v0_T-B0_n*B0_T-n1*v1_n*v1_T-B1_n*B1_T = 0

**Energy conservation** : 1 equation

In [ ]:
# 0.5*(v0_n**2+v0_T**2)*v0_n+2.5*p0*v0_n-B0_T*(B0_n*v0_T-B0_T*v0_n)-0.5*(v1_n**2+v1_T**2)*v1_n+2.5*p1*v1_n-B1_T*(B1_n*v1_T-B1_T*v1_n) = 0

Using the Maxwell-Faraday and the HT frame, energy conservation can be simplified as

In [ ]:
# 0.5*(v0_n**2+v0_T**2)*v0_n+2.5*p0*v0_n-0.5*(v1_n**2+v1_T**2)*v1_n+2.5*p1*v1_n = 0

We then define the cost function which depends on the 7 components vectors :
`x = (n1, v0_n, v0_T, v1_T, p1, B1_n, B1_T)`
with the `args` vector defined as `args = (n0,  v0_n, p0, B0_n, B0_T)`

In [ ]:
def cost(x, *args):
    n1, v0_T, v1_n, v1_T, p1, B1_n, B1_T = x
    n0, v0_n, p0, B0_n, B0_T = args
    
    return [n0*v0_n-n1*v1_n,
            n0*v0_n**2+p0+0.5*(B0_T**2-B0_n**2)-(n1*v1_n**2+p1+0.5*(B1_T**2-B1_n**2)),
            n0*v0_n*v0_T-B0_n*B0_T-(n1*v1_n*v1_T-B1_n*B1_T),
            v0_n+v0_T+5*p0/n0-(v1_n+v1_T+5*p1/n1),
            v0_n*B0_T-B0_n*v0_T,
            v1_n*B1_T-B1_n*v1_T,
            B0_n-B1_n
           ]

In [ ]:
from scipy.optimize import fsolve

In [ ]:
v0_n = v0_n_span[0]

# x0 = (0.1, 0.0, 2.0, 0.5, 0.8, 0.1, 0.5)
#      n1, v0_T, v1_n, v1_T,   p1, B1_n, B1_T
x0 = (0.01,  0.03,  5.1,  4.4,  0.0,  0.9,  0.8)

args=(n0, v0_n, p0, B0_n, B0_T)

In [ ]:
root = fsolve(cost, x0, args=args)

In [ ]:
cost(root, *args)

In [ ]:
n1, v0_T, v1_n, v1_T, p1, B1_n, B1_T = root

theta1 = np.arctan2(B1_T, B1_n)

In [ ]:
gamma = 5./3.

# ___ magnetic field :
B0 = np.sqrt(B0_n**2+B0_T**2)
B1 = np.sqrt(B1_n**2+B1_T**2)

# ___ intermediate velocity :
v0_i = B0_n/np.sqrt(n0)
v1_i = B1_n/np.sqrt(n1)

# ___ Alfven velocity :
v0_A = B0/np.sqrt(n0)
v1_A = B1/np.sqrt(n1)

# ___ sound speed :
c0_s = np.sqrt(gamma*p0/n0)
c1_s = np.sqrt(gamma*p1/n1)

# ___ slow mode :
v0_s = np.sqrt(0.5*(v0_A**2+c0_s**2-np.sqrt((v0_A**2+c0_s**2)**2-4.0*v0_A**2*c0_s**2*(np.cos(theta0))**2)))
v1_s = np.sqrt(0.5*(v1_A**2+c1_s**2-np.sqrt((v1_A**2+c1_s**2)**2-4.0*v1_A**2*c1_s**2*(np.cos(theta1))**2)))

# ___ fast mode :
v0_f = np.sqrt(0.5*(v0_A**2+c0_s**2+np.sqrt((v0_A**2+c0_s**2)**2-4.0*v0_A**2*c0_s**2*(np.cos(theta0))**2)))
v1_f = np.sqrt(0.5*(v1_A**2+c1_s**2+np.sqrt((v1_A**2+c1_s**2)**2-4.0*v1_A**2*c1_s**2*(np.cos(theta1))**2)))

In [ ]:
print("density  : %5.2f   ---  %5.2f" % (n0, n1))
print("U normal : %5.2f   ---  %5.2f" % (v0_n, v1_n))
print("U tang   : %5.2f   ---  %5.2f" % (v0_T, v1_T))
print("pressure : %5.2f   ---  %5.2f" % (p0, p1))
print("B normal : %5.2f   ---  %5.2f" % (B0_n, B1_n))
print("B tang   : %5.2f   ---  %5.2f" % (B0_T, B1_T))

In [ ]:
print("upstream sound speed            : %5.2f" % c0_s)
print("upstream Alfven velocity        : %5.2f" % v0_A)
print("upstream intermediate velocity  : %5.2f" % v0_i)
print("upstream slow velocity          : %5.2f" % v0_s)
print("upstream fast velocity          : %5.2f" % v0_f)

In [ ]:
print("upstream sound speed            : %5.2f" % c0_s)
print("upstream Alfven velocity        : %5.2f" % v0_A)
print("upstream intermediate velocity  : %5.2f" % v0_i)
print("upstream slow velocity          : %5.2f" % v0_s)
print("upstream fast velocity          : %5.2f" % v0_f)

In [ ]:
from scipy.optimize import least_squares

In [ ]:
v0_n = v0_n_span[0]
#       n1,  v0_T, v1_n, v1_T,   p1, B1_n, B1_T
x0 = (0.01,  0.03,  5.1,  4.4,  0.0,  0.9,  0.8)
args=(n0, v0_n, p0, B0_n, B0_T)

In [ ]:
sol = least_squares(cost, x0, args=args, bounds=([0, np.inf]))

In [ ]:
sol

In [ ]:
n1, v0_T, v1_n, v1_T, p1, B1_n, B1_T = sol.x

In [ ]:
theta1 = np.arctan2(B1_T, B1_n)

In [ ]:
gamma = 5./3.

# ___ magnetic field :
B0 = np.sqrt(B0_n**2+B0_T**2)
B1 = np.sqrt(B1_n**2+B1_T**2)

# ___ intermediate velocity :
v0_i = B0_n/np.sqrt(n0)
v1_i = B1_n/np.sqrt(n1)

# ___ Alfven velocity :
v0_A = B0/np.sqrt(n0)
v1_A = B1/np.sqrt(n1)

# ___ sound speed :
c0_s = np.sqrt(gamma*p0/n0)
c1_s = np.sqrt(gamma*p1/n1)

# ___ slow mode :
v0_s = np.sqrt(0.5*(v0_A**2+c0_s**2-np.sqrt((v0_A**2+c0_s**2)**2-4.0*v0_A**2*c0_s**2*(np.cos(theta0))**2)))
v1_s = np.sqrt(0.5*(v1_A**2+c1_s**2-np.sqrt((v1_A**2+c1_s**2)**2-4.0*v1_A**2*c1_s**2*(np.cos(theta1))**2)))

# ___ fast mode :
v0_f = np.sqrt(0.5*(v0_A**2+c0_s**2+np.sqrt((v0_A**2+c0_s**2)**2-4.0*v0_A**2*c0_s**2*(np.cos(theta0))**2)))
v1_f = np.sqrt(0.5*(v1_A**2+c1_s**2+np.sqrt((v1_A**2+c1_s**2)**2-4.0*v1_A**2*c1_s**2*(np.cos(theta1))**2)))

In [ ]:
print("density  : %5.2f   ---  %5.2f" % (n0, n1))
print("U normal : %5.2f   ---  %5.2f" % (v0_n, v1_n))
print("U tang   : %5.2f   ---  %5.2f" % (v0_T, v1_T))
print("pressure : %5.2f   ---  %5.2f" % (p0, p1))
print("B normal : %5.2f   ---  %5.2f" % (B0_n, B1_n))
print("B tang   : %5.2f   ---  %5.2f" % (B0_T, B1_T))

In [ ]:
from scipy.optimize import least_squares

In [ ]:
v0_n_span = np.linspace(0.05, 2, 40)

#       n1,  v0_T, v1_n, v1_T,   p1, B1_n, B1_T
x0 = (0.01,  0.03,  5.1,  4.4,  0.0,  0.9,  0.8)

for v0_n, i in enumerate(v0_n_span):
    args=(n0, v0_n, p0, B0_n, B0_T)
    sol = least_squares(cost, x0, args=args, bounds=([0, np.inf]))
    n1, v0_T, v1_n, v1_T, p1, B1_n, B1_T = sol.x
    x0 = sol.x

In [ ]:
from scipy.optimize import fsolve
import numpy as np
def func(x):
    return [x[0] * np.cos(x[1]) - 4,
            x[1] * x[0] - x[1] - 5]

In [ ]:
root = fsolve(func, [1, 1])

In [ ]:
root

In [ ]:
np.isclose(func(root), [0.0, 0.0])  # func(root) should be almost 0.0.

In [ ]:
from scipy.optimize import fsolve
import numpy as np
def func(x, *args):
    x0, x1 = x
    a, b = args
    return [x0 * np.cos(x1) - a,
            x1 * x0 - x1 - b]

In [ ]:
x0 = (1, 1)
args = (4, 5)
root = fsolve(func, x0, args=args)

In [ ]:
root